In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import awkward
import numpy as np
import matplotlib.pyplot as plot

In [3]:
noise_data_x = generate_distribution(events = 5000, peak = 0.0, width = 10.0, height = 1.0, correlation = 0.0, width_factor = 1.0)
noise_data_y = generate_distribution(events = 5000, peak = 0.0, width = 10.0, height = 1.2, correlation = 0.0, width_factor = 1.0)
noise_data_z = generate_distribution(events = 5000, peak = 0.0, width = 10.0, height = 0.8, correlation = 0.0, width_factor = 1.0)

signal_data_x = generate_distribution(events = 1500, peak = 20.0, width = 10.0, height = 3.0, correlation = 0.4, width_factor = 1.0)
signal_data_y = generate_distribution(events = 1500, peak = 20.0, width = 10.0, height = 3.0, correlation = 0.4, width_factor = 1.0)
signal_data_z = generate_distribution(events = 1500, peak = 20.0, width = 10.0, height = 3.0, correlation = 0.4, width_factor = 1.0)

noise_data_x = noise_data_x[0].tolist()
noise_data_y = noise_data_y[0].tolist()
noise_data_z = noise_data_z[0].tolist()

signal_data_x = signal_data_x[0].tolist()
signal_data_y = signal_data_y[0].tolist()
signal_data_z = signal_data_z[0].tolist()

f:\program files (x86)\python 3.7\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: covariance is not positive-semidefinite.
  # This is added back by InteractiveShellApp.init_path()


In [4]:

# Combine our lists our lists:

data_x = noise_data_x + signal_data_x
data_y = noise_data_y + signal_data_y
data_z = noise_data_z + signal_data_z
noise_list = [0] * 5000
signal_list = [1] * 1500
soaring = noise_list + signal_list 

# Construct a pandas dataframe with all our data:

data = {'var_1':data_x, 'var_2':data_y, 'var_3':data_z, 'sn':soaring}
data = pd.DataFrame(data)

In [5]:
input_data = data.drop('sn', axis=1)
output_data = data['sn']

In [64]:
input_train, input_test, output_train, output_test = train_test_split(input_data, output_data)

scaler = StandardScaler()

scaler.fit(input_train)

input_train = scaler.transform(input_train)
input_test = scaler.transform(input_test)

neural_network = MLPClassifier(hidden_layer_sizes=(50,50), activation='tanh', max_iter=500)

In [65]:
neural_network.fit(input_train, output_train)

MLPClassifier(activation='tanh', hidden_layer_sizes=(50, 50), max_iter=500)

In [66]:
predictions = neural_network.predict(input_test)

In [67]:
print(confusion_matrix(output_test,predictions))
print(classification_report(output_test,predictions))

[[1234    0]
 [   0  391]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1234
           1       1.00      1.00      1.00       391

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



In [69]:
print(input_train)

[[-0.72456416 -0.95885676 -0.47871387]
 [-0.19230635  0.34954655 -1.22553547]
 [-0.6013945  -1.2759914   0.03312678]
 ...
 [ 1.61000917  1.20691074  1.94415955]
 [ 0.2894575  -0.23897526 -1.13865026]
 [ 1.84773025  1.57034862  2.25096794]]


In [2]:
def generate_distribution(events, peak, width, height, correlation, width_factor):
    x_interval = np.array([peak - (width/2), peak + (width/2)])
    y_interval = np.array([0, height])
    vector = np.vstack((x_interval, y_interval)).T

    means = lambda x, y : [x.mean(), y.mean()]
    stdvs = lambda x, y : [x.std() / width_factor, y.std() / width_factor]
    covs = [[stdvs(x_interval, y_interval)[0]**2, stdvs(x_interval, y_interval)[0] * stdvs(x_interval, y_interval)[1] * correlation],
            [stdvs(x_interval, y_interval)[1] * stdvs(x_interval, y_interval)[1] * correlation, stdvs(x_interval, y_interval)[1]**2]]

    generated_data = np.random.multivariate_normal(means(x_interval, y_interval), covs, events).T

    return generated_data